# High performance pandas

´´´py

mask = (x > -5) & (y < -5)

df[df[mask]]

´´´ 

under the hood

tmp = (x > -5)
tmp2 = (y < -5)
mask = tmp & tmp2

use pd.eval(""), df.query() -> does element wise operations with numexpr

In [2]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.349239,2.062269,1.246887,1.416798,1.724651,1.761034,-2.158490,0.112998,-0.587206,0.373564,...,1.068903,1.322456,-1.380553,-1.406844,0.856732,1.689389,-0.765759,-0.035172,0.176108,-1.741075
1,0.105774,0.137098,0.855511,0.077213,-1.590874,-0.136935,-1.478960,0.284396,-0.327495,0.901577,...,-1.208790,0.215260,-1.720975,-0.349857,1.174802,0.756123,-2.258696,-0.202094,1.216593,0.647309
2,1.467677,-0.878403,-1.266643,0.339208,0.456551,0.612364,-0.751221,0.023366,-0.074493,-0.896726,...,1.293291,-1.688794,1.199850,1.487935,1.740142,-1.415321,-0.025019,0.129832,-1.256933,-0.916065
3,1.671586,-1.825365,0.050428,-0.169996,0.458321,0.191399,-0.899041,0.017428,0.103148,0.573247,...,-1.158119,-1.666393,-1.017965,-0.426975,0.612912,1.143963,-0.536787,1.643401,-0.189474,0.590137
4,0.222874,0.874992,-0.172349,-2.169909,0.426589,1.936706,0.126375,1.064419,-0.005930,0.532871,...,-0.231095,1.997672,1.848771,1.020419,0.573314,0.074456,0.865796,-1.151702,-0.979249,0.093984


In [3]:
df1.shape

(1000000, 100)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [5]:
%timeit df1 + df2 + df3 + df4
%timeit pd.eval("df1+df2+df3+df4")

557 ms ± 57.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
249 ms ± 5.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [7]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,1,4,2
1,1,3,5
2,1,5,2
3,3,1,5
4,2,3,2
5,2,3,4


In [8]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace = True)
rolls

,Die1,Die2,Die3,Sum
0,1,4,2,7
1,1,3,5,9
2,1,5,2,8
3,3,1,5,9
4,2,3,2,7
5,2,3,4,9


In [9]:
high = 10
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,1,4,2,7,False
1,1,3,5,9,False
2,1,5,2,8,False
3,3,1,5,9,False
4,2,3,2,7,False
5,2,3,4,9,False


In [10]:
# traditional way to filter from a dataframe
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum


## Query
- filter using query

In [11]:
rolls.query("Sum > @high & Die1 == 4")

,Die1,Die2,Die3,Sum


In [13]:
df_os = pd.read_csv("athlete_events.csv")
df_os

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [16]:
df_os[df_os["NOC"] == "SWE"].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [18]:
df_os.query("NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [19]:
%timeit df_os[df_os["NOC"] == "SWE"]
%timeit df_os.query("NOC == 'SWE'")

12 ms ± 96 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.11 ms ± 74.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit df_os[df_os["Height"] > 180]
%timeit df_os.query("Height > 180")

8.48 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
11.2 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [27]:
df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [29]:
%timeit df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")]
%timeit df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

21.7 ms ± 419 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.9 ms ± 28.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
